In [1]:
import anytree.search
import matplotlib.pyplot as plt
import pandas as pd
import treelib
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains
import time
from anytree import Node, AnyNode, NodeMixin,RenderTree, AbstractStyle, ContStyle, AsciiStyle
from anytree.importer import JsonImporter
from collections import deque
import asyncio
import numpy as np
import os
import plotly as plt
from googleapiclient.discovery import build
import dataframe_image as dfi


In [2]:
importer = JsonImporter()

right_wing_trees = []
left_wing_trees = []

right = os.listdir("D:/Data Science/Comp_396/Project/Tree_results (Right-wing)")
left = os.listdir("D:/Data Science/Comp_396/Project/Tree_results (Left-wing)")

for file in right:
    with open("D:/Data Science/Comp_396/Project/Tree_results (Right-wing)/"+ file, 'r') as outfile:
        data = outfile.read()
        root = importer.import_(data)
        right_wing_trees.append(RenderTree(root))

for file in left:
    with open("D:/Data Science/Comp_396/Project/Tree_results (Left-wing)/"+ file, 'r') as outfile:
        data = outfile.read()
        #data = json.load(data)
        root = importer.import_(data)
        left_wing_trees.append(RenderTree(root))

channels_df = pd.read_csv("D:\Data Science\Comp_396\Project\channel_review.csv")

In [3]:
#remove BBC and CNN from the trees since they are not politically left-wing channels but are more Mainstream
to_remove_left = []
for t in left_wing_trees:
    if(t.node.video['content creator'] in ('BBC') or t.node.video['content creator'] in ('CNN')):
        to_remove_left.append(t)

for x in [0, 1]:
    #there are duplicates of the items we wish to remove to remove them twice
    for i in left_wing_trees:
        if i in to_remove_left:
            left_wing_trees.remove(i)

In [4]:
#
to_remove_right = []
for t in right_wing_trees:
    if(t.node.video['content creator'] in ('PowerfulJRE') or t.node.video['content creator'] in ('JRE Clips')):
        to_remove_right.append(t)

for x in [0, 1]:
    #there are duplicates of the items we wish to remove to remove them twice
    for i in right_wing_trees:
        if i in to_remove_right:
            right_wing_trees.remove(i)

# Right-Wing Tree analysis

In [5]:
# create a dataframe of right wing results for right wing tree
columns_right = ['AntiSJW', 'Conspiracy', 'PartisanRight', 'ReligiousConservative', 'WhiteIdentitarian', 'Libertarian', 'MRA', 'Provocateur', 'Total']
depth = [i for i in range(0, 6)]
df_right = pd.DataFrame(data = ([ 0 for i in columns_right ] for x in range(0, 6)), columns=columns_right)

outer = 0

for root in right_wing_trees:
    outer = outer + 1
    #creators[root.node.id] = []
    #t.video['content creator']
    for t in anytree.LevelOrderIter(root.node):
        if not (t.video == None):
            if not (t.video['content creator'] == None):

                #only looking for the exact name
                if (channels_df['CHANNEL_TITLE'].str.contains('^'+ t.video['content creator'] +'$').sum() > 0):
                    # now check what the type of the channels found were
                    c = channels_df.loc[channels_df['CHANNEL_TITLE'].str.contains('^'+ t.video['content creator'] +'$', case=False, na=False)]

                    bool = False
                    for tags in columns_right:
                        if (c['REVIEWER_TAGS'].str.contains(tags).sum() > 0) or (c['TAGS'].str.contains(tags).sum() > 0):
                            bool = True
                            #add a + 1 in the column
                            df_right.at[t.depth, tags] = df_right.at[t.depth, tags] + 1
                    if bool:
                        df_right.at[t.depth, 'Total'] = df_right.at[t.depth, 'Total'] + 1

    print(f'Outer: {outer}/{len(right_wing_trees)}')

Outer: 1/116
Outer: 2/116
Outer: 3/116


C:\Users\mikad\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Outer: 4/116
Outer: 5/116
Outer: 6/116
Outer: 7/116
Outer: 8/116
Outer: 9/116
Outer: 10/116
Outer: 11/116
Outer: 12/116
Outer: 13/116
Outer: 14/116
Outer: 15/116
Outer: 16/116
Outer: 17/116
Outer: 18/116
Outer: 19/116
Outer: 20/116
Outer: 21/116
Outer: 22/116
Outer: 23/116
Outer: 24/116
Outer: 25/116
Outer: 26/116
Outer: 27/116
Outer: 28/116
Outer: 29/116
Outer: 30/116
Outer: 31/116
Outer: 32/116
Outer: 33/116
Outer: 34/116
Outer: 35/116
Outer: 36/116
Outer: 37/116
Outer: 38/116
Outer: 39/116
Outer: 40/116
Outer: 41/116
Outer: 42/116
Outer: 43/116
Outer: 44/116
Outer: 45/116
Outer: 46/116
Outer: 47/116
Outer: 48/116
Outer: 49/116
Outer: 50/116
Outer: 51/116
Outer: 52/116
Outer: 53/116
Outer: 54/116
Outer: 55/116
Outer: 56/116
Outer: 57/116
Outer: 58/116
Outer: 59/116
Outer: 60/116
Outer: 61/116
Outer: 62/116
Outer: 63/116
Outer: 64/116
Outer: 65/116
Outer: 66/116
Outer: 67/116
Outer: 68/116
Outer: 69/116
Outer: 70/116
Outer: 71/116
Outer: 72/116
Outer: 73/116
Outer: 74/116
Outer: 75/11

In [6]:
d = [0 for i in range(0, 6)]

for t in right_wing_trees:
    for nodes in anytree.LevelOrderIter(t.node):
        if(nodes.depth < 6):
            d[nodes.depth] = d[nodes.depth] + 1

for i in range(0, len(d)):
    df_right.loc[i] = df_right.loc[i]/d[i]

In [7]:
df_styled_right = df_right.style.background_gradient() #adding a gradient based on values in cell

In [8]:
dfi.export(df_right,"df_right_styled.png")

OSError: [Errno 22] Invalid argument: 'df_right_styled.png'

In [9]:
df_right

,AntiSJW,Conspiracy,PartisanRight,ReligiousConservative,WhiteIdentitarian,Libertarian,MRA,Provocateur,Total
0,0.801724,0.146552,0.931034,0.310345,0.068966,0.344828,0.103448,0.362069,0.931034
1,0.045977,0.002874,0.218391,0.014368,0.000000,0.008621,0.002874,0.014368,0.250000
2,0.030918,0.000966,0.119807,0.005797,0.000000,0.002899,0.000000,0.004831,0.147826
3,0.013834,0.000988,0.073452,0.003953,0.000000,0.001976,0.000000,0.003623,0.086627
4,0.009550,0.000920,0.049361,0.002531,0.000000,0.001956,0.000000,0.001496,0.059027
5,0.000160,0.000000,0.001520,0.000120,0.000000,0.000000,0.000000,0.000480,0.002160


# Left-Wing

In [10]:
columns_left = ['SocialJustice', 'Socialist', 'PartisanLeft', 'AntiWhiteness', 'LGBT', 'Total']

depth = [i for i in range(0, 6)]
df_left = pd.DataFrame(data = ([ 0 for i in columns_left ] for x in range(0, 6)), columns=columns_left)

outer = 0

for root in left_wing_trees:
    outer = outer + 1
    #creators[root.node.id] = []
    #t.video['content creator']
    for t in anytree.LevelOrderIter(root.node):
        if not (t.video == None):
            if not (t.video['content creator'] == None):

                #only looking for the exact name
                try:
                    if (channels_df['CHANNEL_TITLE'].str.contains('^'+ t.video['content creator'] +'$').sum() > 0):
                        # now check what the type of the channels found were
                        c = channels_df.loc[channels_df['CHANNEL_TITLE'].str.contains('^'+ t.video['content creator'] +'$', case=False, na=False)]

                        bool = False
                        for tags in columns_left:
                            if (c['REVIEWER_TAGS'].str.contains(tags).sum() > 0) or (c['TAGS'].str.contains(tags).sum() > 0):
                                bool = True
                                #add a + 1 in the column
                                df_left.at[t.depth, tags] = df_left.at[t.depth, tags] + 1
                        if bool:
                            df_left.at[t.depth, 'Total'] = df_left.at[t.depth, 'Total'] + 1
                except:
                    continue
    print(f'Outer: {outer}/{len(left_wing_trees)}')

Outer: 1/129
Outer: 2/129


C:\Users\mikad\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Outer: 3/129
Outer: 4/129
Outer: 5/129
Outer: 6/129
Outer: 7/129
Outer: 8/129
Outer: 9/129
Outer: 10/129
Outer: 11/129
Outer: 12/129
Outer: 13/129
Outer: 14/129
Outer: 15/129
Outer: 16/129
Outer: 17/129
Outer: 18/129
Outer: 19/129
Outer: 20/129
Outer: 21/129
Outer: 22/129
Outer: 23/129
Outer: 24/129
Outer: 25/129
Outer: 26/129
Outer: 27/129
Outer: 28/129
Outer: 29/129
Outer: 30/129
Outer: 31/129
Outer: 32/129
Outer: 33/129
Outer: 34/129
Outer: 35/129
Outer: 36/129
Outer: 37/129
Outer: 38/129
Outer: 39/129
Outer: 40/129
Outer: 41/129
Outer: 42/129
Outer: 43/129
Outer: 44/129
Outer: 45/129
Outer: 46/129
Outer: 47/129
Outer: 48/129
Outer: 49/129
Outer: 50/129
Outer: 51/129
Outer: 52/129
Outer: 53/129
Outer: 54/129
Outer: 55/129
Outer: 56/129
Outer: 57/129
Outer: 58/129
Outer: 59/129
Outer: 60/129
Outer: 61/129
Outer: 62/129
Outer: 63/129
Outer: 64/129
Outer: 65/129
Outer: 66/129
Outer: 67/129
Outer: 68/129
Outer: 69/129
Outer: 70/129
Outer: 71/129
Outer: 72/129
Outer: 73/129
Outer: 74/129

In [11]:
x = [0 for i in range(0, 6)]

for t in left_wing_trees:
    for nodes in anytree.LevelOrderIter(t.node):
        if(nodes.depth < 6):
            x[nodes.depth] = x[nodes.depth] + 1

for i in range(0, len(x)):
    df_left.loc[i] = df_left.loc[i]/x[i]

In [12]:
df_left.style.set_caption("Results of trees with left-wing seed videos")


,SocialJustice,Socialist,PartisanLeft,AntiWhiteness,LGBT,Total
0,1.000000,0.790698,0.844961,0.000000,0.062016,1.000000
1,0.330749,0.149871,0.325581,0.000000,0.023256,0.354005
2,0.148438,0.061632,0.158854,0.000000,0.007812,0.176215
3,0.081602,0.032219,0.092743,0.000000,0.003914,0.101174
4,0.047320,0.013608,0.064124,0.000000,0.001546,0.067526
5,0.006532,0.001095,0.007556,0.000000,0.000071,0.007662


In [13]:
df_right.style.set_caption("Results of trees with right-wing seed videos")


,AntiSJW,Conspiracy,PartisanRight,ReligiousConservative,WhiteIdentitarian,Libertarian,MRA,Provocateur,Total
0,0.801724,0.146552,0.931034,0.310345,0.068966,0.344828,0.103448,0.362069,0.931034
1,0.045977,0.002874,0.218391,0.014368,0.000000,0.008621,0.002874,0.014368,0.250000
2,0.030918,0.000966,0.119807,0.005797,0.000000,0.002899,0.000000,0.004831,0.147826
3,0.013834,0.000988,0.073452,0.003953,0.000000,0.001976,0.000000,0.003623,0.086627
4,0.009550,0.000920,0.049361,0.002531,0.000000,0.001956,0.000000,0.001496,0.059027
5,0.000160,0.000000,0.001520,0.000120,0.000000,0.000000,0.000000,0.000480,0.002160


In [18]:
dfi.export(df_left,"df_left.png")

# Isolating AntiSJW

In [14]:
anti_SJW = []

for r in right_wing_trees:
    if (channels_df.loc[channels_df['CHANNEL_TITLE'].str.contains('^'+ r.node.video['content creator'] +'$', case=False, na=False)]['REVIEWER_TAGS'].str.contains('AntiSJW').sum() > 0):
        anti_SJW.append(r)

In [15]:
df_antiSJW = [0 for i in range(0, 6)]
outer = 0

for root in anti_SJW:
    outer = outer + 1

    for t in anytree.LevelOrderIter(root.node):
        if not (t.video == None):
            if not (t.video['content creator'] == None):
                #only looking for the exact name
                if (channels_df['CHANNEL_TITLE'].str.contains('^'+ t.video['content creator'] +'$').sum() > 0):
                    # now check what the type of the channels found were
                    c = channels_df.loc[channels_df['CHANNEL_TITLE'].str.contains('^'+ t.video['content creator'] +'$', case=False, na=False)]

                    if (c['REVIEWER_TAGS'].str.contains('AntiSJW').sum() > 0) or (c['TAGS'].str.contains('AntiSJW').sum() > 0):
                            bool = True
                            #add a + 1 in the column
                            df_antiSJW[t.depth] = df_antiSJW[t.depth] + 1
    print(f'Outer: {outer}/{len(anti_SJW)}')

Outer: 1/93
Outer: 2/93
Outer: 3/93
Outer: 4/93
Outer: 5/93
Outer: 6/93
Outer: 7/93
Outer: 8/93
Outer: 9/93
Outer: 10/93
Outer: 11/93
Outer: 12/93
Outer: 13/93
Outer: 14/93
Outer: 15/93
Outer: 16/93
Outer: 17/93
Outer: 18/93
Outer: 19/93
Outer: 20/93
Outer: 21/93
Outer: 22/93
Outer: 23/93
Outer: 24/93
Outer: 25/93
Outer: 26/93
Outer: 27/93
Outer: 28/93
Outer: 29/93
Outer: 30/93
Outer: 31/93
Outer: 32/93
Outer: 33/93
Outer: 34/93
Outer: 35/93
Outer: 36/93
Outer: 37/93
Outer: 38/93
Outer: 39/93
Outer: 40/93
Outer: 41/93
Outer: 42/93
Outer: 43/93
Outer: 44/93
Outer: 45/93
Outer: 46/93
Outer: 47/93
Outer: 48/93
Outer: 49/93
Outer: 50/93
Outer: 51/93
Outer: 52/93
Outer: 53/93
Outer: 54/93
Outer: 55/93
Outer: 56/93
Outer: 57/93
Outer: 58/93
Outer: 59/93
Outer: 60/93
Outer: 61/93
Outer: 62/93
Outer: 63/93
Outer: 64/93
Outer: 65/93
Outer: 66/93
Outer: 67/93
Outer: 68/93
Outer: 69/93
Outer: 70/93
Outer: 71/93
Outer: 72/93
Outer: 73/93
Outer: 74/93
Outer: 75/93
Outer: 76/93
Outer: 77/93
Outer: 7

In [16]:
y = [0 for i in range(0, 6)]

for t in anti_SJW:
    for nodes in anytree.LevelOrderIter(t.node):
        if(nodes.depth < 6):
            y[nodes.depth] = y[nodes.depth] + 1

for i in range(0, len(y)):
    df_antiSJW[i] = df_antiSJW[i]/y[i]

In [17]:
print(df_antiSJW)

[1.0, 0.053763440860215055, 0.035024154589371984, 0.013991769547325103, 0.010079193664506839, 0.00020032051282051281]


In [35]:
list_1 = []

for t in range(0, len(left_wing_trees)):
    if(t == 64):
        break
    i = left_wing_trees[t]
    list_1.append([i.node.video['title'], i.node.video['content creator']])
    
df_list_left_1 = pd.DataFrame(data=list_1, columns=['Title', 'Content Creator'])

In [36]:
list_2 = []

for t in range(64, len(left_wing_trees)):

    i = left_wing_trees[t]
    list_2.append([i.node.video['title'], i.node.video['content creator']])
    
df_list_left_2 = pd.DataFrame(data=list_2, columns=['Title', 'Content Creator'])

In [38]:
dfi.export(df_list_left_1,"D:\Data Science\Comp_396\Project\df_list_left_1.png", max_rows=-1)

dfi.export(df_list_left,"D:\Data Science\Comp_396\Project\df_list_left_2.png", max_rows=-1)